In [2]:
import pickle
from scipy.io import loadmat
import numpy as np
import pandas as pd
from time import time as t
import matplotlib.pyplot as plt
#!pip install tqdm
from tqdm import tqdm

from SpykeTorch import functional, snn, utils, visualization

In [3]:
ttfs_spikes_train = pd.read_pickle(r'ttfs_spikes_data/ttfs_spikes_train.p')
# print(ttfs_spikes_train.shape) # (index, time-frame, frequency-band)

### Split the data into the overlapping section

In [4]:
kernel_size = (6,40)
stride = 1
n_sections = 9
size_conv_section = 4
size_inpt_section = 9

i = 0
shape = ttfs_spikes_train.shape
split_data = np.zeros((n_sections, shape[0], size_inpt_section, shape[2]))
for section in range(n_sections):
    split_data[section, :, :, :] = ttfs_spikes_train[:, i:i+size_inpt_section, :]
    i += size_conv_section

(9, 2464, 9, 40)
(9, 2464, 9, 40)


In [5]:
def one_hot2spikes(data):
    """
    Computes one-hot encoding from data
    Returns: [samples, time-frames, frequency bands, time-points]
    """
    n_spiketime_bins = np.max(data).astype(int) + 1
    spikes = np.zeros(list(data.shape)+[n_spiketime_bins])
    for i, sample in enumerate(data):
        for j, tf in enumerate(sample):
            spikes[i, j] = np.eye(n_spiketime_bins)[tf]

    return spikes

data_one_hot = one_hot2spikes(split_data.astype(int))
print(data_one_hot.shape)
shape = data_one_hot.shape

(9, 2464, 9, 40, 32)


### Apply convolution and pooling to the sections

In [8]:
data_channels = np.expand_dims(data_one_hot, 1)  # The channel dimension is the second dimension of a tensor


conv = snn.Convolution(in_channels=40, out_channels=40, kernel_size=(6,40))
pool = snn.Pooling(kernel_size=(4,40), stride=4)

stdp = snn.STDP(conv_layer=conv, learning_rate=(0.004, 0.003))
